# Example: Custom `MoveStrategy`: RepEx-Shoot-Repex

One of the powerful features in OpenPathSampling is that it is very easy to develop new Monte Carlo movers for path space. This example shows how easy it is to try out a new type of move. The particular move we use here can be easily described as a simple combination of existing moves, so we don't even need to define a new `PathMover` subclass. We just define a custom `MoveStrategy` that creates the desired `PathMover`, and use that directly.

The idea implemented here is pretty simple. Our standard path movers treat shooting and replica exchange separately, and each move is a single shooting (one ensemble) or a single replica exchange (swap one pair). But maybe you could get better replica exchange behavior by trying all the replica exchange moves, and then trying all the shooting moves. Note that, to satisfy detailed balance, you really have to do all the replica exchange moves, then all the shooting moves, then all the replica exchange moves in the reverse order from before. To measure how this affects travel in replica space, we'll use the replica round trip time (normalized to the total number of shooting moves per ensemble).

In [1]:
import openpathsampling as paths
import numpy as np

# Set up the simulation

## Set up the engine

In [2]:
import openpathsampling.engines.toy as toys
pes = (toys.OuterWalls([1.0,1.0], [0.0,0.0]) + 
       toys.Gaussian(-0.7, [12.0, 0.5], [-0.5, 0.0]) +
       toys.Gaussian(-0.7, [12.0, 0.5], [0.5, 0.0]))

topology = toys.Topology(n_spatial=2, masses=[1.0, 1.0], pes=pes)

engine = toys.Engine(options={'integ': toys.LangevinBAOABIntegrator(dt=0.02, temperature=0.1, gamma=2.5),
                              'n_frames_max': 5000,
                              'n_steps_per_frame': 10},
                     topology=topology)

template = toys.Snapshot(coordinates=np.array([[0.0, 0.0]]),
                         velocities=np.array([[0.0, 0.0]]),
                         engine=engine)

## Set up CV and volumes (states, interfaces)

In [3]:
# states are volumes in a CV space: define the CV
def xval(snapshot):
    return snapshot.xyz[0][0]

cv = paths.FunctionCV("xval", xval)

stateA = paths.CVDefinedVolume(cv, float("-inf"), -0.5).named("A")
stateB = paths.CVDefinedVolume(cv, 0.5, float("inf")).named("B")
interfaces_AB = paths.VolumeInterfaceSet(cv, float("-inf"), [-0.5, -0.4, -0.3, -0.2, -0.1, 0.0])

## Set up network

In [4]:
network = paths.MISTISNetwork([(stateA, interfaces_AB, stateB)])

## Define a custom strategy

In [5]:
import openpathsampling.analysis.move_strategy as strategies # TODO: handle this better
# example: custom subclass of `MoveStrategy`
class RepExShootRepExStrategy(strategies.MoveStrategy):
    _level = strategies.levels.GROUP
    # we define an init function mainly to set defaults for `replace` and `group`
    def __init__(self, ensembles=None, group="repex_shoot_repex", replace=True, network=None):
        super(RepExShootRepExStrategy, self).__init__(
            ensembles=ensembles, group=group, replace=replace
        )
            
    def make_movers(self, scheme):
        # if we replace, we remove these groups from the scheme.movers dictionary
        if self.replace:
            repex_movers = scheme.movers.pop('repex')
            shoot_movers = scheme.movers.pop('shooting')
        else:
            repex_movers = scheme.movers['repex']
            shoot_movers = scheme.movers['shooting']
        # combine into a list for the SequentialMover
        mover_list = repex_movers + shoot_movers + list(reversed(repex_movers))
        combo_mover = paths.SequentialMover(mover_list)
        return [combo_mover]

## Create two move schemes: Default and Custom

In [6]:
default_scheme = paths.DefaultScheme(network, engine)

In [7]:
custom_scheme = paths.DefaultScheme(network, engine)
custom_scheme.append(RepExShootRepExStrategy())

# Get initial conditions

In [8]:
initial_samples = paths.FullBootstrapping(transition=network.sampling_transitions[0],
                                          snapshot=template,
                                          engine=engine).run()

DONE! Completed Bootstrapping cycle step 446 in ensemble 6/6.


In [9]:
transition = network.sampling_transitions[0]
minus_sample = network.minus_ensembles[0].populate_minus_ensemble(
    partial_traj=initial_samples[transition.ensembles[0]].trajectory,
    minus_replica_id=-1,
    engine=engine
)
initial_samples = initial_samples.apply_samples(minus_sample)

In [10]:
initial_samples.sanity_check()

In [11]:
print "Default Scheme:", default_scheme.initial_conditions_report(initial_samples)
print "Custom Scheme:", custom_scheme.initial_conditions_report(initial_samples)

Default Scheme: No missing ensembles.
No extra ensembles.

Custom Scheme: No missing ensembles.
No extra ensembles.



# Run each of the simulations

In [12]:
n_steps = default_scheme.n_steps_for_trials(
    mover=default_scheme.movers['shooting'],
    n_attempts=600
)
n_steps = int(n_steps)+1
print n_steps

1170


In [13]:
default_storage = paths.Storage("default_scheme.nc", "w")

In [14]:
default_calc = paths.PathSampling(
    storage=default_storage,
    sample_set=initial_samples,
    move_scheme=default_scheme
)

In [15]:
default_calc.run(n_steps)

Working on Monte Carlo cycle number 1170.
DONE! Completed 1170 Monte Carlo cycles.


In [16]:
n_steps = custom_scheme.n_steps_for_trials(
    mover=custom_scheme.movers['repex_shoot_repex'],
    n_attempts=100
)
n_steps = int(n_steps)+1
print n_steps

421


In [17]:
custom_storage = paths.Storage("custom_scheme.nc", "w")

In [18]:
custom_calc = paths.PathSampling(
    storage=custom_storage,
    sample_set=initial_samples,
    move_scheme=custom_scheme
)

In [19]:
custom_calc.run(n_steps)

Working on Monte Carlo cycle number 421.
DONE! Completed 421 Monte Carlo cycles.


# Analyze the results